<a href="https://colab.research.google.com/github/Bearbrice/maDLC_MM_hands2/blob/main/examples/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590444465547-SHXODUII311HEE407IL6/ke17ZwdGBToddI8pDm48kE4VnnB9_j2k1VP236ADqAFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxQg9Vf0owGyf3dhfDKy8SxMujaKmp2B54Sb3VS1rO76Whq-cUhHVuKFlGUXsU9tJk/ezgif.com-video-to-gif.gif?format=1500w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub.

Mathis et al, in prep. <- please note, we are providing this toolbox as an early access release; more feeatures and details will be released with the forthcoming paper.




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!#pip install deeplabcut
!pip install 'deeplabcut[gui]'==2.2rc3
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

     |████████████████████████████████| 696kB 13.3MB/s 
     |████████████████████████████████| 184kB 27.2MB/s 
     |████████████████████████████████| 13.1MB 210kB/s 
     |████████████████████████████████| 3.1MB 33.8MB/s 
     |████████████████████████████████| 21.6MB 14.0MB/s 
     |████████████████████████████████| 20.0MB 40.2MB/s 
     |████████████████████████████████| 112kB 43.0MB/s 
     |████████████████████████████████| 29.2MB 107kB/s 
     |████████████████████████████████| 296kB 38.8MB/s 
     |████████████████████████████████| 9.5MB 20.8MB/s 
     |████████████████████████████████| 552kB 33.0MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=b5c03f3fd807ced85ceefba8f9ba79d950f54502279e5880b80f2f868bc1e499
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=de5

In [1]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"
import deeplabcut

TensorFlow 1.x selected.
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


# GitHub

In [2]:
# Clone the github repository
!git clone -l -s git://github.com/Bearbrice/maDLC_MM_hands2.git
!ls

Cloning into 'maDLC_MM_hands2'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 173 (delta 14), reused 160 (delta 8), pack-reused 0
Receiving objects: 100% (173/173), 45.48 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (14/14), done.
maDLC_MM_hands2  sample_data


## YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Now, you can do this within COLAB! Simply navigate in the left panel to your project folder, double click on the config.yaml file, and you will see it load on the right! Edit the first part of your path, to match:

 /content/drive/My Drive/yourProjectFolderName


In [3]:
# PLEASE EDIT THIS:
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/maDLC_MM_hands2/Inference_videos'] #Enter the list of videos or folder to analyze.
videofile_path


#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/maDLC_MM_hands2/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
path_config_file
#This creates a path variable that links to your google drive project

'/content/maDLC_MM_hands2/maDLC_MM_hands-Brice-2021-07-08/config.yaml'

## Create a multi-animal training dataset:
### You must do this step inside of Colab:

- Reminder: you must connect EVERY bodypart in a skeleton before you run this step! You should OVER CONNECT all the parts for training. You can do this in the GUI before you upload your project (and in the future this will be automatically done).

See docs for crucial details on how to do this effciently: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#b-configure-the-project-

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589256735280-SCN7CROSJNJWCDS6EK5T/ke17ZwdGBToddI8pDm48kB08p9-rNkpPD7A3fw8YFjZZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PIno0kSvzOWihTW1zp8-1-7mzYxUQjsVr2n3nmNdVcso4/bodyparts-skeleton.png?format=1000w)
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589410182515-9SJO9MML6CNCXBAWQ6Z6/ke17ZwdGBToddI8pDm48kJ1oJoOIxBAgRD2ClXVCmKFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxBw7VlGKDQO2xTcc51Yv6DahHgScLwHgvMZoEtbzk_9vMJY_JknNFgVzVQ2g0FD_s/ezgif.com-video-to-gif+%2811%29.gif?format=750w)

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [5]:
# ATTENTION:

#Note, you must run this. If your images are smaller than 400 by 400, please make these numbers smaller.
deeplabcut.cropimagesandlabels(path_config_file, size=(400, 400), userfeedback=False)

#if you labeled on Windows, please set the windows2linux=True:
deeplabcut.create_multianimaltraining_dataset(path_config_file, windows2linux=True)

  0%|          | 0/20 [00:00<?, ?it/s]

/content/maDLC_MM_hands2/maDLC_MM_hands-Brice-2021-07-08/labeled-data/Pil01_cropped  already exists!


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


RuntimeError: ignored

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. 
#Typically, you want to train to 50,000 iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

#which shuffle do you want to train?
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100, saveiters=2500, maxiters=20000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True,c_engine=False)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

In [ ]:
#Cross-validate for Animal Assembly & Tracking:
deeplabcut.evaluate_multianimal_crossvalidate(
            path_config_file,
            Shuffles=[shuffle],
            edgewisecondition=True,
            leastbpts=1,
            init_points=20,
            n_iter=100,
            target='rpck_train',
        )

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
print("Start Analyzing my video(s)!")
scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)

## The steps below work on a single video at a time.
- Here you can create a video to check the pose estimation detection quality! If this looks good, proceed to tracklet conversions with the interactive GUI (ouside of COLAB for now), or if you know your optimal parameters, you can automate this and run the additional steps shown in a few cells down.

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
##  don't look good, don't proceed with tracking - add more data to your training set and re-train!

#let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/My Drive/yourproject/videos/demo.mp4'

from deeplabcut.utils import auxiliaryfunctions
scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(path_config_file, shuffle, trainFraction=0)
print("scorename is: "+scorername)

deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

#Again, if this does not look perfect, do not proceed! Retrain with more diverse data.

## Convert Detections to Tracklets:

- The idea is that you test and adapt hyperparameters for tracking outside of COLAB. Once you have good parameters, this can be automated on future videos. Shown here!

- I.e., instead of always doing an interactive parameter setting step, you can simply convert tracklets to .h5 files using these parameters (see GitHub for more info).

In [ ]:
#assemble tracklets:
#read the docs: which tracker to test out (you can run this many times to try multiple):
tracktype= 'box' #box, skeleton, ellipse

deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)

## Now you should manually verify the tracks and correct them if needed! [currently only working outside of COLAB]

In [ ]:
''' here is the code you would need:
os.environ["DLClight"]="False"
import deeplabcut

#ATTENTION:
picklefile = '/...._10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/yourVIDEO.mp4'
#if you want occlusions filled in, tell us how many frames to fill in, i.e. if there is a gap in data:
framestofill = 0. #note, put "0" if you want ALL gaps filled!

%matplotlib inline

from deeplabcut import refine_tracklets
TrackletManager, TrackletVisualizer = refine_tracklets(path_config_file, 
                                                          picklefile, 
                                                          Specific_videofile, 
                                                          min_swap_frac=0,
                                                          min_tracklet_frac=0, 
                                                          max_gap=framestofill)
'''

## Let's assume you have great tracking parameters, and you want to analyze a full set of videos:

In [ ]:
#^^^^^^^^^You do NOT neeed to run if you hit "save" in the GUI ^^^^^^^^^^
#this is just if you want to run the same parameters over a set of videos!

# You need to point to your pickle file, please "copy path" from the folder to the left (right click, copy path)
picklefile = '/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemoDLC_resnet50_mwmMar31shuffle1_22000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemo.mp4'

deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)

## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=True, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)